# 🏪 Minimart POS Database Design & Core Queries (Group Project Deliverable)
## 1. Business Domain and Key Activities
**Domain:** Minimart Point-of-Sale (POS) and Inventory Management System. <br>
**Objective:** To design and implement a system for cashiers to process sales and managers to handle inventory, user, and promotion management.

### ⚠️ Important Note
This notebook contains SQL-style examples for academic demonstration.  
In our real system, we do **not** write raw SQL manually. Instead, we use:

- **FastAPI** for the application layer  
- **SQLModel** (built on SQLAlchemy) as the ORM  
- **Alembic** for schema migrations  
- **PostgreSQL** as the database

SQLModel automatically generates all SQL queries at runtime.  
Therefore, the SQL shown here is only an illustration of what actions the
application performs behind the scenes.

## 2. Database Schema Definitions (Python SQLModel and SQL DDL)
The schema is based on the Python `SQLModel` definitions in the `backend/app/models` directory. All foreign keys and constraints are enforced at the database level.

In [ ]:
-- SQL Data Definition Language (DDL) Equivalent for PostgreSQL

-- User Table
CREATE TABLE "user" (
    uid VARCHAR(255) NOT NULL PRIMARY KEY,
    email VARCHAR(255) NOT NULL UNIQUE,
    username VARCHAR(255) NOT NULL UNIQUE,
    name VARCHAR(255) NOT NULL,
    hashed_password VARCHAR(255) NOT NULL,
    role VARCHAR(255) NOT NULL DEFAULT 'cashier',
    is_active BOOLEAN NOT NULL DEFAULT true,
    created_at TIMESTAMP WITHOUT TIME ZONE NOT NULL
);

-- Role Tables (Cashier, Manager)
CREATE TABLE cashier (
    employee_id VARCHAR(255) NOT NULL PRIMARY KEY,
    FOREIGN KEY(employee_id) REFERENCES "user" (uid)
);
CREATE TABLE manager (
    admin_id UUID NOT NULL PRIMARY KEY,
    FOREIGN KEY(admin_id) REFERENCES "user" (uid)
);

-- MembershipTier Table
CREATE TABLE membershiptier (
    rank_name VARCHAR(255) NOT NULL PRIMARY KEY,
    min_spent NUMERIC(10, 2) NOT NULL,
    max_spent NUMERIC(10, 2) DEFAULT NULL,
    discount_rate NUMERIC(5, 2) NOT NULL,
    benefits VARCHAR(255) DEFAULT NULL,
    CHECK (min_spent >= 0),
    CHECK ((max_spent IS NULL) OR (max_spent > min_spent)),
    CHECK (discount_rate BETWEEN 0 AND 100)
);

-- Member Table
CREATE TABLE member (
    member_id SERIAL NOT NULL PRIMARY KEY,
    phone VARCHAR(255) NOT NULL UNIQUE,
    points_balance INTEGER NOT NULL DEFAULT 0,
    total_spent NUMERIC(10, 2) NOT NULL DEFAULT 0.00,
    membership_rank VARCHAR(255) NOT NULL DEFAULT 'Bronze',
    discount_rate NUMERIC(5, 2) NOT NULL DEFAULT 3.00,
    registration_date DATE NOT NULL,
    FOREIGN KEY(membership_rank) REFERENCES membershiptier (rank_name),
    CHECK (points_balance >= 0),
    CHECK (total_spent >= 0)
);

-- Promotion Table
CREATE TABLE promotion (
    promotion_id SERIAL NOT NULL PRIMARY KEY,
    promotion_name VARCHAR(255) NOT NULL,
    discount_type VARCHAR(255) NOT NULL,
    discount_value NUMERIC(10, 2) NOT NULL,
    start_date DATE NOT NULL,
    end_date DATE NOT NULL,
    is_active BOOLEAN NOT NULL DEFAULT true,
    CHECK (discount_type IN ('PERCENTAGE','FIXED')),
    CHECK (discount_value > 0),
    CHECK (end_date >= start_date)
);

-- Product Table
CREATE TABLE product (
    product_id SERIAL NOT NULL PRIMARY KEY,
    barcode VARCHAR(255) NOT NULL UNIQUE,
    name VARCHAR(255) NOT NULL,
    cost_price NUMERIC(10, 2) NOT NULL,
    selling_price NUMERIC(10, 2) NOT NULL,
    stock_quantity INTEGER NOT NULL DEFAULT 0,
    min_stock INTEGER NOT NULL DEFAULT 10,
    promotion_id INTEGER DEFAULT NULL,
    FOREIGN KEY(promotion_id) REFERENCES promotion (promotion_id),
    CHECK (selling_price >= cost_price),
    CHECK (stock_quantity >= 0),
    CHECK (min_stock > 0)
);

-- Transaction Table
CREATE TABLE "transaction" (
    transaction_id SERIAL NOT NULL PRIMARY KEY,
    transaction_date TIMESTAMP WITHOUT TIME ZONE NOT NULL,
    employee_id VARCHAR(255) NOT NULL,
    member_id INTEGER DEFAULT NULL,
    subtotal NUMERIC(10, 2) NOT NULL,
    product_discount NUMERIC(10, 2) NOT NULL DEFAULT 0.00,
    membership_discount NUMERIC(10, 2) NOT NULL DEFAULT 0.00,
    total_amount NUMERIC(10, 2) NOT NULL,
    payment_method VARCHAR(255) NOT NULL,
    FOREIGN KEY(employee_id) REFERENCES cashier (employee_id),
    FOREIGN KEY(member_id) REFERENCES member (member_id),
    CHECK (payment_method IN ('Cash','Card','QR Code')),
    CHECK (total_amount = subtotal - membership_discount)
);

-- TransactionItem Table
CREATE TABLE transactionitem (
    transaction_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    unit_price NUMERIC(10, 2) NOT NULL,
    discount_amount NUMERIC(10, 2) NOT NULL DEFAULT 0.00,
    line_total NUMERIC(10, 2) NOT NULL,
    PRIMARY KEY (transaction_id, product_id),
    FOREIGN KEY(transaction_id) REFERENCES "transaction" (transaction_id),
    FOREIGN KEY(product_id) REFERENCES product (product_id),
    CHECK (quantity > 0),
    CHECK (line_total = (quantity * unit_price) - discount_amount)
);

## 3. Specific Actions/Queries (14 Queries)
These parameterized queries correspond to critical operations within the application. User input is denoted by `<parameter_name>`.

1. User Sign Up (Create New User)
- Description: Creates a new user record (cashier or manager) after performing validation checks (duplicate email/username, manager secret code).

In [ ]:
INSERT INTO "user" (
  uid, email, username, name, hashed_password, role, is_active, created_at
) VALUES (
  '<new_uuid>', '<email>', '<username>', '<name>', '<hashed_password>', '<role_cashier_or_manager>', TRUE, NOW()
) RETURNING uid;

2. User Sign In (Authentication)

- Description: Retrieves a user record by matching the identifier (email or username) to validate credentials.

In [ ]:
SELECT
  uid, hashed_password
FROM "user"
WHERE
  email = '<email_or_username_identifier>' OR username = '<email_or_username_identifier>'
LIMIT 1;

3. Create Product (Manager Action)

- Description: Creates a new product, ensuring unique barcode and checking the core business 
- constraint: selling price must be greater than or equal to the cost price.

In [ ]:
INSERT INTO product (
  barcode, name, cost_price, selling_price, stock_quantity, min_stock, brand, category
) VALUES (
  '<new_barcode>', '<new_name>', <cost_price>, <selling_price>, <initial_stock>, <min_stock_threshold>, '<brand_or_null>', '<category_or_null>'
) RETURNING product_id;

4. Search Product by Query (POS/Inventory Lookup)

- Description: Retrieves a list of products matching a query term against multiple fields (name, brand, category, barcode).

In [ ]:
SELECT
  product_id, name, barcode, selling_price, stock_quantity, promotion_id, brand, category
FROM product
WHERE
  name ILIKE '%<query_term>%' OR
  brand ILIKE '%<query_term>%' OR
  category ILIKE '%<query_term>%' OR
  barcode ILIKE '%<query_term>%' 
LIMIT 50;

5. Update Product Stock/Details (Manager Action)

- Description: Updates any product field (including stock_quantity or selling_price), ensuring the price constraint (selling_price >= cost_price) is still met after the update.

In [ ]:
-- This performs a partial update of non-primary/non-foreign key fields
UPDATE product
SET 
  name = COALESCE(<new_name_or_null>, name),
  selling_price = COALESCE(<new_price_or_null>, selling_price),
  stock_quantity = COALESCE(<new_stock_qty_or_null>, stock_quantity),
  promotion_id = <new_promo_id_or_null>
WHERE product_id = <product_id_to_update>;

6. Delete Product (Manager Action)

- Description: Permanently removes a product record.

In [ ]:
DELETE FROM product
WHERE product_id = <product_id_to_delete>;

7. Create Promotion (Manager Action)

- Description: Creates a new promotion record after validating discount type and date logic.

In [ ]:
INSERT INTO promotion (
  promotion_name, discount_type, discount_value, start_date, end_date, is_active
) VALUES (
  '<promo_name>', '<PERCENTAGE_or_FIXED>', <discount_value>, '<start_date_YYYY-MM-DD>', '<end_date_YYYY-MM-DD>', TRUE
) RETURNING promotion_id;

8. Update Promotion (Manager Action)

- Description: Updates a promotion's details, including re-validating complex rules involving date and percentage values.

In [ ]:
UPDATE promotion
SET
  promotion_name = COALESCE(<new_name_or_null>, promotion_name),
  discount_value = COALESCE(<new_value_or_null>, discount_value),
  is_active = COALESCE(<new_active_status_or_null>, is_active)
WHERE promotion_id = <promotion_id_to_update>;

9. Delete Promotion (Unlink Products & Delete)

- Description: Deletes a promotion. It first explicitly sets promotion_id = NULL for all associated products to avoid foreign key errors (data integrity).

In [ ]:
BEGIN;
  -- Step 1: Unlink from products
  UPDATE product
  SET promotion_id = NULL
  WHERE promotion_id = <promotion_id_to_delete>;
  
  -- Step 2: Delete the promotion record
  DELETE FROM promotion
  WHERE promotion_id = <promotion_id_to_delete>;
COMMIT;

10. List Employees (Manager Report)

- Description: Retrieves and sorts employee data for the Manager dashboard, prioritizing managers over cashiers for display.

In [ ]:
SELECT
  uid, email, name, role
FROM "user"
WHERE
  role = '<role_filter_manager_or_cashier>' -- Optional WHERE clause
ORDER BY 
  CASE WHEN role = 'manager' THEN 0 ELSE 1 END, name ASC;

11. Identify Low Stock Products (Dashboard KPI)

- Description: Query to find all products where the current stock_quantity is below the min_stock threshold, typically used for alerts and reports.

In [ ]:
SELECT
  product_id, name, stock_quantity, min_stock
FROM product
WHERE
  stock_quantity < min_stock
ORDER BY
  (stock_quantity - min_stock) ASC -- Most deficient first
LIMIT 10;

12. Insert Transaction Record (Checkout Step 1)

- Description: Creates the master transaction record as the first step of the atomic checkout process.

In [ ]:
-- SQL Equivalent (A12: Insert Transaction)
INSERT INTO "transaction" (
  employee_id, member_id, subtotal, product_discount, membership_discount, total_amount, payment_method, transaction_date
) VALUES (
  '<employee_uid>', <member_id_or_null>, <subtotal_after_prod_disc>, <total_prod_disc>, <member_disc>, <final_total>, '<payment_method>', NOW()
) RETURNING transaction_id;

13. Insert Transaction Line Item (Checkout Step 2)

- Description: Inserts each product sold into the transactionitem table.

In [ ]:
INSERT INTO transactionitem (
  transaction_id, product_id, quantity, unit_price, discount_amount, line_total
) VALUES (
  <new_tx_id>, <product_id>, <quantity>, <unit_price>, <item_discount>, <line_total>
);

14. Update Member Tier (Loyalty Progression)

- Description: Checks the member's total spending against all membership tiers and updates their rank if they qualify for a higher tier.

In [ ]:
-- Step 1: Select the new tier (if any)
SELECT
  rank_name, discount_rate
FROM membershiptier
WHERE
  min_spent <= (SELECT total_spent FROM member WHERE member_id = <member_id>)
ORDER BY min_spent DESC
LIMIT 1;

-- Step 2: Update the member (executed if a new tier is returned)
UPDATE member
SET
  membership_rank = '<new_rank_name>',
  discount_rate = <new_discount_rate>
WHERE member_id = <member_id> AND membership_rank != '<new_rank_name>';